# CR ML Road Map


- Step 1:
    - Transfrom Mock Data from Text File to Numpy Dataframe 
- Step 2:
    - ReCalculate Normalized Factor for Mock Data
- Step 3:
    - Machine Learning
        - using whole data to train
        - retrain the model with the data in 6 $\sigma$ CL region 
- Step 4:
   - Create pseudo data from pseudo experiment based on the uncertainty from experimental data
- Step 5:
   - Using retrained model to predict the parameter for pseudo data
- Step 6:
    - Send predicted parameters back to GALPROP for MC simulation
- Step 7:
    - TBA
- Step 8:
    - TBA
- Step 9:
    - TBA

## Import Packages

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
# basic python package
import importlib
import numpy as np
import time
import logging
importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# python ploting packages
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib.collections import LineCollection
from matplotlib import cm


# self-define classes
from script import CR_ML_Class as CR
from script import load_mock_data as LD


# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
print("Tensorflow Version is {}".format(tf.__version__))
print("Keras Version is {}".format(tf.keras.__version__))

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)
!nvidia-smi

Tensorflow Version is 2.4.1
Keras Version is 2.4.0
1 Physical GPUs, 1 Logical GPU
Fri Aug  6 03:24:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:E1:00.0 Off |                  N/A |
| 42%   51C    P2   114W / 350W |   3519MiB / 24268MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
      

## Transfrom Mock Data from Text File to Numpy Dataframe 

- Data Description:


    - parameter : propagation and source parameters 
        numpy shape: (# of mock data, 14)
            #parameter: original parameter 
            #new_parameter: recalculate the normal factor and Ap 

            raw1=D_0, 
            raw2=\delta, 
            raw3=z_h, 
            raw4=v_A, 
            raw5=\eta, 
            raw6=A_p, 
            raw7=\nu_1, 
            raw8=\nu_2, 
            raw9=log10(R_{br,1}), 
            raw10=\nu_3, 
            raw11=log10(R_{br,2}), 
            raw12=N_{Li}, 
            raw13=N_{Be}, 
            raw14=N_{O}


    - data: Mock data
        numpy shape: (# of mock data, 84, 6)
            84 means there are 84 energy bins from 1.000e-03 to 1.105e+05
            6 means there are the spectrum for E, Li, Be, B, C, O
            #data_0: original mocak data
            #modify_data_0: reshape data accroding to recalculated parameters


     - chi: chi-square 
        numpy shape: (# of mock data)
            #new_chi: chi-square from "modify_data"

In [2]:
%%time
text_Data_path = "../Data/Text_Mock_data/"

# mockdata_1 = CR.Mock_Data_to_NumpyArray(text_Data_path + "res_3x_1.txt")
mockdata_1 = CR.Mock_Data_to_NumpyArray(text_Data_path + "return_4.txt")
origin_parameter, data, chisq = mockdata_1.parameter, mockdata_1.spectrum, mockdata_1.chisq

logging.info("Data Shape for 'parameter': {}".format(origin_parameter.shape))
logging.info("Data Shape for 'data': {}".format(data.shape))
logging.info("Data Shape for 'chisq': {}".format(chisq.shape))

INFO:root:Fri Aug 06 03:29:21 2021
INFO:root:Now loading...
INFO:root:Total data: 30000
INFO:root:Time consumption : 0.4290 min
INFO:root:Data Shape for 'parameter': (30000, 15)
INFO:root:Data Shape for 'data': (2520000, 11)
INFO:root:Data Shape for 'chisq': (30000,)


CPU times: user 25.5 s, sys: 743 ms, total: 26.2 s
Wall time: 26.2 s


## ReCalculate Normalized Factor for Mock Data

- 1: Using   
    `CR.Mock_Data_Rescale(parameter, parameter, data)`   
    to split 'data' into Li, Be, B, C and O `spectra`.  
    Note that we put the same 'parameter' here because we have not get new normalized factor yet
    
    - `spectra`: Mock data
        numpy shape: (# of mock data, 84, 6)
            84 means there are 84 energy bins from 1.000e-03 to 1.105e+05
            6 means there are the spectrum for E, Li, Be, B, C, O
            #data_0: original mocak data
            #modify_data_0: reshape data accroding to recalculated paramete

In [3]:
%%time
spectra_data = CR.Mock_Data_Rescale(origin_parameter=origin_parameter, new_parameter=origin_parameter, spectrum=data, usedata=False)
logging.info("Data Shape for 'spectra_data': {}".format(spectra_data.data.shape))
logging.info("There are {} mock data.".format(spectra_data.data.shape[0]))
logging.info("For each mock data, there are {} energy bins.".format(spectra_data.data.shape[1]))
logging.info("{} corresponding to E, Li, Be, B, C and O.".format(spectra_data.data.shape[2]))

INFO:root:Data Shape for 'spectra_data': (30000, 84, 6)
INFO:root:There are 30000 mock data.
INFO:root:For each mock data, there are 84 energy bins.
INFO:root:6 corresponding to E, Li, Be, B, C and O.


CPU times: user 179 ms, sys: 2.18 ms, total: 181 ms
Wall time: 178 ms


- 2: Using  
`CR.ReCalculateAp(spectra_data.data)`   
  to recalculate Ap

In [4]:
%%time
importlib.reload(CR)
new_Ap = CR.ReCalculateAp(spectra_data.data[:100]) 
new_Ap.GetBestAp()

INFO:root:Fri Aug 06 03:29:47 2021
INFO:root:Finding best Ap
INFO:root:=====START=====
100%|██████████| 100/100 [00:00<00:00, 101.06it/s]
INFO:root: Time Cost for this Step : 0.0166 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0167 min


CPU times: user 1.01 s, sys: 7.51 ms, total: 1.01 s
Wall time: 1.01 s


- 3: Using   
`CR.ReCalculateN((spectra_data.new_parameter,spectra_data.data,ap=new_Ap.ap_5))`  
to recalculate normalized factor ($N_{Li}$, $N_{Be}$ and $N_{O}$)

In [5]:
%%time
importlib.reload(CR)
new_normalized_factor = CR.ReCalculateN(spectra_data.new_parameter[:100],spectra_data.data[:100],ap=new_Ap.ap_5)
new_normalized_factor.GetBestN()

INFO:root:Fri Aug 06 03:29:48 2021
INFO:root:Finding New Normalized Factor
INFO:root:=====START=====
100%|██████████| 100/100 [00:01<00:00, 84.88it/s]
INFO:root: Time Cost for this Step : 0.0197 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0198 min


CPU times: user 1.18 s, sys: 12.9 ms, total: 1.2 s
Wall time: 1.19 s


- 4: Using   
    `CR.New_Parameter(spectra_data.new_parameter,new_normalized_factor.new_factor,ap_5=new_Ap.ap_5).new_parameter`
    to get new parameter array

In [6]:
%%time
new_parameter = CR.New_Parameter(spectra_data.new_parameter[:100],new_normalized_factor.new_factor,ap_5=new_Ap.ap_5).new_parameter

INFO:root:Fri Aug 06 03:29:49 2021
INFO:root:Time Cost : 0.0000 min


CPU times: user 1.32 ms, sys: 748 µs, total: 2.07 ms
Wall time: 1.48 ms


- 5: Put `origin_parameter` and `new_parameter` back into  
    `CR.Mock_Data_Rescale(parameter, new_parameter, spectra_data.data,usedata = True)`  
    to get `new parameter` array, `new data` array and new $\chi^2$`. 

In [7]:
%%time
new_spectra_data = CR.Mock_Data_Rescale(origin_parameter[:100], new_parameter, spectra_data.data[:100], usedata = True)

parameter = new_spectra_data.new_parameter
data = new_spectra_data.data

chisq = CR.Calculate_Chi_Square(data=data,usedata=True) 
chi = chisq.chi_square()

logging.info("Data Shape for 'parameter': {}".format(parameter.shape))
logging.info("Data Shape for 'data': {}".format(data.shape))
logging.info("Data Shape for 'chi': {}".format(chi.shape))

INFO:root:Fri Aug 06 03:29:49 2021
INFO:root:Fit the Spectrum.
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Calculate Chi-Square.
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:

INFO:root: Total Time Consumption : 0.0002 min
INFO:root:Data Shape for 'parameter': (100, 14)
INFO:root:Data Shape for 'data': (100, 84, 6)
INFO:root:Data Shape for 'chi': (100,)


CPU times: user 15 ms, sys: 1.99 ms, total: 17 ms
Wall time: 14.8 ms


- 6: Using  
    `CR.Select_Sample(chi_para, chi_data, chi_sele,1).Sample()`  
    to seperate data into different CL region.

In [8]:
%%time
importlib.reload(CR)
chi_para, chi_data, chi_sele = parameter, data, chi

para_1_sigma, data_1_sigma, _ =  CR.Select_Sample(chi_para, chi_data, chi_sele,1).Sample()
para_2_sigma, data_2_sigma, _ =  CR.Select_Sample(chi_para, chi_data, chi_sele,2).Sample()
para_3_sigma, data_3_sigma, _ =  CR.Select_Sample(chi_para, chi_data, chi_sele,3).Sample()
para_4_sigma, data_4_sigma, _ =  CR.Select_Sample(chi_para, chi_data, chi_sele,4).Sample()
para_5_sigma, data_5_sigma, _ =  CR.Select_Sample(chi_para, chi_data, chi_sele,5).Sample()
para_6_sigma, data_6_sigma, _ =  CR.Select_Sample(chi_para, chi_data, chi_sele,6).Sample()

INFO:root:There are 5 data in the 1 σ region.
INFO:root:Time consumption : 0.0000 min
INFO:root:There are 9 data in the 2 σ region.
INFO:root:Time consumption : 0.0000 min
INFO:root:There are 20 data in the 3 σ region.
INFO:root:Time consumption : 0.0000 min
INFO:root:There are 35 data in the 4 σ region.
INFO:root:Time consumption : 0.0000 min
INFO:root:There are 46 data in the 5 σ region.
INFO:root:Time consumption : 0.0000 min
INFO:root:There are 65 data in the 6 σ region.
INFO:root:Time consumption : 0.0000 min


CPU times: user 11.9 ms, sys: 2.28 ms, total: 14.2 ms
Wall time: 11.3 ms


# Machine Learning

- 1: Using  
     `CR.Mock_Data_Processing(parameter=parameter, data=data, usedata = True)`
     and
     `.Train_Test_split(splitrate = 0.1, split = True)`   
     to whitening data and split into training and test data set with the ratio 9:1.

In [9]:
%%time
importlib.reload(CR)
data_processing = CR.Mock_Data_Processing(parameter=parameter, data=data, usedata = True)
data_processing.Train_Test_split(splitrate = 0.1, split = True)

input_train, input_test = data_processing.input_train, data_processing.input_test
source_train, source_test = data_processing.source_train, data_processing.source_test



INFO:root:Fri Aug 06 03:29:49 2021
INFO:root:Prepare Ratio
INFO:root:Fri Aug 06 03:29:49 2021
INFO:root:Whitening
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0000 min
INFO:root:random split traning sample and test sample, 10% for test
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Shape for training Input: (90, 8, 84)
INFO:root:Shape for  testing Input: (10, 8, 84)
INFO:root:Shape for training Target: (90, 10)
INFO:root:Shape for  testing Target: (10, 10)
INFO:root:Time Cost : 0.0003 min


CPU times: user 19 ms, sys: 3 ms, total: 22 ms
Wall time: 18.7 ms


- 2:Using   
    `ML.ML_Training(input_train,input_test,source_train,source_test,EPOCH=10, save_path="./")`
    to train a model with whole CL region.

In [10]:
%%time
from script import ML_Training as ML
importlib.reload(ML)

ML.ML_Training(input_train,input_test,source_train,source_test,EPOCH=10, save_path="./")

INFO:root:
-----------------------------------------------------------
""python3""
 ML_Training.py 
 Usage: ML_Training(input_train,input_test,source_train,source_test,EPOCH=100,save_path="save_path") 
        Trained Model will be stroed in "Model" directory 
 Usage: Load_ML_ML_Training(input_train,input_test,source_train,source_test,model_path,EPOCH=250,BATCH=256,save_path="save_path")
        Load Model for snd training 
        Trained Model will be stroed in "Model" directory 


-----------------------------------------------------------

INFO:root:
-----------------------------------------------------------
""python3""
 ML_Training.py 
 Usage: ML_Training(input_train,input_test,source_train,source_test,EPOCH=100,save_path="save_path") 
        Trained Model will be stroed in "Model" directory 
 Usage: Load_ML_ML_Training(input_train,input_test,source_train,source_test,model_path,EPOCH=250,BATCH=256,save_path="save_path")
        Load Model for snd training 
        Trained Model 

Model: "Sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_input (Conv1D)        (None, 8, 512)            43520     
_________________________________________________________________
Conv1D_1 (Conv1D)            (None, 8, 512)            262656    
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 8, 256)            131328    
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 8, 256)            65792     
_________________________________________________________________
Conv1D_4 (Conv1D)            (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4, 128)            0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

INFO:root:accuracy: 0.60000
INFO:root:mse: 0.00711
INFO:root:mae: 0.05640
INFO:root:mape: 22.43865
INFO:root:Time consumption : 0.1388 min


CPU times: user 5.35 s, sys: 2.41 s, total: 7.75 s
Wall time: 8.34 s


- 3:Using   
    `ML.Load_ML_Training(input_train,input_test,source_train,source_test,model_path="./Model/CR_ML.h5", EPOCH = 250, BATCH = 256, save_path="./")`  
    to retrain a model in 6 $\sigma$ CL region.

In [11]:

data_processing = CR.Mock_Data_Processing(parameter=para_6_sigma, data=data_6_sigma, usedata = True)
data_processing.Train_Test_split(splitrate = 0.1, split = True)

input_train, input_test = data_processing.input_train, data_processing.input_test
source_train, source_test = data_processing.source_train, data_processing.source_test


ML.Load_ML_Training(input_train,input_test,source_train,source_test, model_path="./Model/CR_ML.h5", EPOCH = 10, BATCH = 256, save_path="./")

INFO:root:Fri Aug 06 03:29:58 2021
INFO:root:Prepare Ratio
INFO:root:Fri Aug 06 03:29:58 2021
INFO:root:Whitening
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0001 min
INFO:root:random split traning sample and test sample, 10% for test
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Shape for training Input: (58, 8, 84)
INFO:root:Shape for  testing Input: (7, 8, 84)
INFO:root:Shape for training Target: (58, 10)
INFO:root:Shape for  testing Target: (7, 10)
INFO:root:Time Cost : 0.0004 min
INFO:root:Fri Aug 06 03:29:58 2021


Model: "Sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_input (Conv1D)        (None, 8, 512)            43520     
_________________________________________________________________
Conv1D_1 (Conv1D)            (None, 8, 512)            262656    
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 8, 256)            131328    
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 8, 256)            65792     
_________________________________________________________________
Conv1D_4 (Conv1D)            (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4, 128)            0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

INFO:root:accuracy: 0.42857
INFO:root:mse: 0.00847
INFO:root:mae: 0.05817
INFO:root:mape: 50.89824
INFO:root:Time consumption : 0.0462 min


## Create pseudo data from pseudo experiment based on the uncertainty from experimental data
- 1.Using   
    `CR.Create_Pseudodata(para_6_sigma,data_6_sigma,chi, LOW = 10 , HIGH = 100000 , number = 2000, index=0).Create_Pseudodata()`  
    to do the pseudo experiment based on the mock data in 6 $\sigma$ CL region.

In [12]:
%%time
importlib.reload(CR)
pseudoexp = CR.Create_Pseudodata(para_6_sigma, data_6_sigma, chi, LOW = 10 , HIGH = 100000 , number = 200, index=0).Create_Pseudodata()


INFO:root:Fri Aug 06 03:30:01 2021
INFO:root:Experimental data are loading.
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0001 min
INFO:root:=====Finish=====
INFO:root:Using whitening data to make pseudodata
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Search The Pack of Spectrum
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Create Pseudodata
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0007 min
INFO:root:=====Finish=====
INFO:root:Recorver to (Spectrum)/(Normal Factor)
INFO:root:=====START=====
INFO:root:Pseudo Normal Factor
INFO:root:        Li     Be     O   
INFO:root:  #    200    200    200  
INFO:root: Max  1.1843 1.0039 1.0541
INFO:root: Min  1.1384 0.9637 1.0145
INFO:root: Ave. 1.1609 0.9911 1.0311
INFO:root: Time Cost for this Step : 0.0070 min
INFO:root:=====Finish=====
INFO:root:Recorver to Spectrum
INFO:ro

CPU times: user 499 ms, sys: 11.5 ms, total: 510 ms
Wall time: 503 ms


- 2.Using  
   `CR.Calculate_Chi_Square(data=pseudodata,usedata=True)`  
    to check $\chi^2$ for pseudodata

In [13]:
%%time

normalfactor = pseudoexp[0]
pseudodata = pseudoexp[1]

chi_pseudo = CR.Calculate_Chi_Square(data=pseudodata,usedata=True)
chi_pseudo = chi_pseudo.chi_square()
index_pseudo = np.argsort(chi_pseudo)
logging.info("There are {} pseudo data.".format(len(chi_pseudo)))
logging.info("minium chi: {}".format(min(chi)))
logging.info("{} of pseudodata in 1 sigma region".format(np.count_nonzero(chi_pseudo < min(chi) + 15.9  )))
logging.info("{} of pseudodata in 2 sigma region".format(np.count_nonzero(chi_pseudo < min(chi) + 24.03  )))
logging.info("{} of pseudodata in 3 sigma region".format(np.count_nonzero(chi_pseudo < min(chi) + 33.20  )))

INFO:root:Fri Aug 06 03:30:01 2021
INFO:root:Fit the Spectrum.
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0001 min
INFO:root:=====Finish=====
INFO:root:Calculate Chi-Square.
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:

INFO:root: Total Time Consumption : 0.0003 min
INFO:root:There are 200 pseudo data.
INFO:root:minium chi: 367.99501
INFO:root:13 of pseudodata in 1 sigma region
INFO:root:28 of pseudodata in 2 sigma region
INFO:root:55 of pseudodata in 3 sigma region


CPU times: user 21 ms, sys: 3.17 ms, total: 24.2 ms
Wall time: 21.4 ms


-3.Using  
   `CR.Mock_Data_Processing(parameter=normalfactor, data=pseudodata, usedata = True)`   
    to whiten pseudo data for prediction

In [16]:
%%time
importlib.reload(CR)
pseudodata_processing = CR.Mock_Data_Processing(parameter=normalfactor, data=pseudodata, usedata = True)
pseudodata_processing.Train_Test_split(splitrate = 0, split = False)

input_train, input_test = pseudodata_processing.input_train, pseudodata_processing.input_test
source_train, source_test = pseudodata_processing.source_train, pseudodata_processing.source_test

INFO:root:Fri Aug 06 03:31:16 2021
INFO:root:Prepare Ratio
INFO:root:Fri Aug 06 03:31:16 2021
INFO:root:Whitening
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Time Cost : 0.0001 min
INFO:root:random split traning sample and test sample, 10% for test
INFO:root:=====START=====
INFO:root: Time Cost for this Step : 0.0000 min
INFO:root:=====Finish=====
INFO:root:Shape for training Input: (200, 8, 84)
INFO:root:Shape for  testing Input: (0, 8, 84)
INFO:root:Shape for training Target: (200, 10)
INFO:root:Shape for  testing Target: (0, 10)
INFO:root:Time Cost : 0.0007 min


CPU times: user 50.9 ms, sys: 3.42 ms, total: 54.3 ms
Wall time: 49.4 ms


## Using retrained model to predict the parameter for pseudo data
- 1.Load model and estimate the parameters for pseudo experiment

In [17]:
%%time
importlib.reload(CR)
model = load_model("./Model/CR_ML_2nd.h5")
prediction_pseudodata = model.predict(input_train)


CPU times: user 447 ms, sys: 43.3 ms, total: 491 ms
Wall time: 480 ms


- 2.Using  
    ` CR.Recovery(prediction_pseudodata).para_recovery`  
    to transform the prediction to real parameter's scale


In [18]:
%%time
prediction_recovery = CR.Recovery(prediction_pseudodata).para_recovery

logging.info("Data Shape for 'prediction_recovery': {}".format(prediction_recovery.shape))

INFO:root:Data Shape for 'prediction_recovery': (200, 10)


CPU times: user 2.6 ms, sys: 0 ns, total: 2.6 ms
Wall time: 2.12 ms


In [16]:
importlib.reload(CR)

help(CR.Create_Pseudodata)

Help on class Create_Pseudodata in module script.CR_ML_Class:

class Create_Pseudodata(builtins.object)
 |  Methods defined here:
 |  
 |  Create_Pseudodata(self)
 |      Usage: 
 |          Create_Pseudodata(parameter, data, chi, LOW = 10 , HIGH = 30 , number = 100, index=0).Create_Pseudodata()
 |      Return:
 |          normalfactor, pseudodata
 |      Item:
 |          Null
 |  
 |  __init__(self, parameter=[], data=[], chi=[], LOW=10, HIGH=30, number=100, index=0)
 |      Usage: 
 |          Create_Pseudodata(parameter, data, chi, LOW = 10 , HIGH = 30 , number = 100, index=0)
 |      Return:
 |          Null
 |      Item:
 |          Null
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

